In [50]:
import networkx as nx
from tqdm import tqdm
import random
from leafer import Leafer
import numpy as np
import pickle

In [87]:
#G = nx.read_edgelist("./IsA_Graph2.edgelist", delimiter=" ", create_using=nx.DiGraph)
G = nx.read_edgelist("ru_isa.edgelist", delimiter="\t", create_using=nx.DiGraph)

In [88]:
l = Leafer(G)
#iterator = l.leafs_generator()

In [89]:
train, test = l.split_train_test(
    generation_depth=1, # до какого уровня в топ. сортировке идти
    p=0.05, # вероятность что подходящий случай уйдет в тест
    p_divide_leafs=0.5, 
    # вероятность что листья поделим пополам трейн-тест
    # а не засунем целый случай в трейн или в тест
    min_to_test_rate=0.5,
    #минимальное количество доли вершин которых не было в 
    #трейне чтобы поделить пополам на трейн-тест
    #то есть если 6\10 вершин были трейне то значит все 10 в трейн
    #если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
    weights=[0.25, 0.3, 0.25, 0.1, 0.1], 
    #веса в соответствии 
    #один ребенок, только листья, не только листья
    #триплеты с 2 родителями, триплеты такие что мидл нода имеет
    # 1 ребенка
                                )
                                

In [90]:
len(train), len(test)

(10370, 423)

In [91]:
train_count = {}
for elem in train:
    if elem['case'] in train_count.keys():
        train_count[elem['case']] += 1

    else:
        train_count[elem['case']] = 1

test_count = {}
for elem in test:
    if elem['case'] in test_count.keys():
        test_count[elem['case']] += 1

    else:
        test_count[elem['case']] = 1

In [92]:
train_count, test_count

({'only_leafs_all': 2127,
  'simple_triplet_grandparent': 1840,
  'leafs_and_no_leafs': 1371,
  'simple_triplet_2parent': 2925,
  'only_child_leaf': 2052,
  'only_leafs_divided': 55},
 {'only_leafs_all': 53,
  'leafs_and_no_leafs': 55,
  'only_leafs_divided': 55,
  'only_child_leaf': 94,
  'simple_triplet_2parent': 78,
  'simple_triplet_grandparent': 88})

In [93]:
name_train = '../Data/train_ru_babel.pickle'
name_test = '../Data/test_ru_babel.pickle'

with open(name_train, 'wb') as handle:
    pickle.dump(train, handle)

with open(name_test, 'wb') as handle:
    pickle.dump(test, handle)


In [42]:
num_leaks = 0
for vertex in l.collector.test_verteces:
    if vertex in l.collector.train_verteces:
        num_leaks += 1

In [43]:
num_leaks

0

In [44]:
num_leaks = 0
for vertex in l.collector.train_verteces:
    if vertex in l.collector.test_verteces:
        num_leaks += 1

In [45]:
num_leaks

0